In [28]:
import cffi

from cffi import FFI
ffi = FFI()
ffi.cdef('int fib(int);')
C = ffi.verify('int fib(int n) { return n<2?n:fib(n-1)+fib(n-2);}')

In [29]:
%timeit -n 10 C.fib(40)

10 loops, best of 3: 919 ms per loop


## PyBind11 + CppImport
save the following file to code.cpp

In [ ]:
<%
cfg['compiler_args'] = ['-std=c++11', '-stdlib=libc++', '-mmacosx-version-min=10.7']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>

namespace py = pybind11;

double fib(int n) {
    return  n < 2?n:fib(n-1)+fib(n-2);
}

PYBIND11_PLUGIN(code) {
    pybind11::module m("code", "auto-compiled c++ extension");
    m.def("fib", fib);
    return m.ptr();
}

In [4]:
import cppimport
code = cppimport.imp("code")

%timeit -n 10 code.fib(40)

10 loops, best of 3: 782 ms per loop


In [24]:
def fib(n):
    return n if n < 2 else fib(n-1) + fib(n-2)

# dynamic programming
hash = [0]*100
hash[:2] = [1, 1]

def fib3(n):
    if n < 2:
        return n
    else:
        if hash[n-2] == 0:
            hash[n-2] = fib3(n-1)
        if hash[n-3] == 0:
            hash[n-3] = fib3(n-2)
            
        hash[n-1] = hash[n-2] + hash[n-3]
        return hash[n-1]

In [26]:
%timeit -n 10 fib3(40)

10 loops, best of 3: 811 ns per loop


In [6]:
%timeit -n 1 fib(40)

1 loop, best of 3: 54.8 s per loop


In [31]:
from numba import jit

@jit
def fib2(n):
    return n if n < 2 else fib2(n-1) + fib2(n-2)

%timeit -n 10 fib2(40)

10 loops, best of 3: 1.12 s per loop


In [30]:
import wrap

%timeit -n 10 wrap.fib(40)

10 loops, best of 3: 735 ms per loop
